# Hist Design Prototype

This is `fill` method in python loop:

In [1]:
import numpy as np
import numba as nb
from hist import Hist
from hist import axis

array = np.random.randn(
    10000,
)
h = Hist.new.Reg(100, -3, 3, name="x", label="x-axis").Double()

In [2]:
# python fill
h.fill(array)
h

Hist(Regular(100, -3, 3, name='x', label='x-axis'), storage=Double()) # Sum: 9973.0 (10000.0 with flow)

## Numba: Hist

To extend the Numba, we first need to create a Hist type `HistType` for `Hist`, and then teach Numba about our type inference additions:

In [3]:
from numba import types
from numba.extending import typeof_impl, as_numba_type, type_callable

# create Numba type
class HistType(types.Type):
    def __init__(self):
        super().__init__(name="Hist")


hist_type = HistType()

# infer values
@typeof_impl.register(Hist)
def typeof_index(val, c):
    return hist_type


# infer annotations
as_numba_type.register(Hist, hist_type)

# infer operations
@type_callable(Hist)
def type_hist(context):
    def typer(axes):
        for ax in axes:
            if not (
                isinstance(ax, hist.axis.Regular)
            ):  # ToDo: Aassumed all are Regular axes
                return typer

    return hist_type

We also need to teach Numba how to actually generate native representation for the new operations:

In [4]:
from numba.core import cgutils
from numba.extending import (
    models,
    register_model,
    make_attribute_wrapper,
    overload_attribute,
    lower_builtin,
    box,
    unbox,
    NativeValue,
)

# define data model
@register_model(HistType)
class HistModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ("bins", types.int32),
            ("lo", types.float64),
            ("hi", types.float64),
        ]
        models.StructModel.__init__(self, dmm, fe_type, members)


# expose attributes, porperties and constructors
make_attribute_wrapper(HistType, "bins", "bins")
make_attribute_wrapper(HistType, "lo", "lo")
make_attribute_wrapper(HistType, "hi", "hi")


@lower_builtin(Hist, types.Integer, types.Float, types.Float)
def impl_h(context, builder, sig, args):
    typ = sig.return_type
    lo, hi, bins = args
    h = cgutils.create_struct_proxy(typ)(context, builder)
    h.lo = lo
    h.hi = hi
    h.bins = bins
    return h._getvalue()


# unbox and box
@unbox(HistType)
def unbox_h(typ, obj, c):
    bins_obj = c.pyapi.object_getattr_string(obj, "bins")
    lo_obj = c.pyapi.object_getattr_string(obj, "lo")
    hi_obj = c.pyapi.object_getattr_string(obj, "hi")
    h = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    h.bins = c.pyapi.float_as_double(bins_obj)
    h.lo = c.pyapi.float_as_double(lo_obj)
    h.hi = c.pyapi.float_as_double(hi_obj)
    c.pyapi.decref(bins_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(h._getvalue(), is_error=is_error)


@box(HistType)
def box_h(typ, val, c):
    h = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    bins_obj = c.pyapi.float_from_double(h.bins)
    lo_obj = c.pyapi.float_from_double(h.lo)
    hi_obj = c.pyapi.float_from_double(h.hi)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Hist))
    res = c.pyapi.call_function_objargs(class_obj, (bins_obj, lo_obj, hi_obj))
    c.pyapi.decref(bins_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    c.pyapi.decref(class_obj)
    return res

In [8]:
@nb.njit
def nb_fill_hist(h):
    h.fill(np.random.randn(10))

In [9]:
reg_ax = axis.Regular(10, -3, 3)
h1 = Hist(reg_ax)

nb_fill_hist(h1)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'fill' of type Hist

File "<ipython-input-8-49c26ac8b7f7>", line 3:
def nb_fill_hist(h):
    h.fill(np.random.randn(10))
    ^

During: typing of get attribute at <ipython-input-8-49c26ac8b7f7> (3)

File "<ipython-input-8-49c26ac8b7f7>", line 3:
def nb_fill_hist(h):
    h.fill(np.random.randn(10))
    ^
